In [1]:
# This code performs a grid search to find the best hyperparameters for a variety of machine learning algorithms on the breast cancer dataset.

# Import necessary libraries
from sklearn import datasets, model_selection, metrics, preprocessing
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

# Load the breast cancer dataset
cancer = datasets.load_breast_cancer()

# Preprocess the dataset
X = cancer['data']
y = cancer['target']
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=0)

# Define the parameters to search over for each algorithm
tuned_parameters = [[{'kernel': ['rbf', 'linear'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]}],
                    [{'max_depth': [10, 100, 1000, 10000], 'min_samples_split': [2, 10, 100], 'min_samples_leaf': [1, 5, 10], 'max_features': ["sqrt", "log2"]}],
                    [{'activation': ['logistic', 'tanh', 'relu'], 'hidden_layer_sizes': [(5,), (10,)], 'max_iter': [200, 1000], 'alpha': [0.0001, 0.0005]}],
                    [{}],  # GaussianNB
                    [{'penalty': ['l1', 'l2'], 'tol': [1e-4, 1e-5], 'max_iter': [10, 100, 1000], 'fit_intercept': [True, False]}],
                    [{'n_neighbors': [5, 10, 20], 'weights': ['uniform', 'distance'], 'algorithm': ['ball_tree', 'kd_tree', 'brute'], 'p': [1, 2, 3]}],
                    [{'n_estimators': [10, 20, 100], 'max_samples': [0.5, 1.0], 'max_features': [0.5, 1.0], 'random_state': [None]}],
                    [{'n_estimators': [10, 20, 100], 'max_features': [0.5, 1.0], 'criterion': ['gini', 'entropy'], 'max_depth': [None, 100, 200]}],
                    [{'n_estimators': [50, 100, 200], 'random_state': [None], 'learning_rate': [1., 0.8, 0.5], 'algorithm': ['SAMME', 'SAMME.R']}],
                    [{'loss': ['deviance', 'exponential'], 'n_estimators': [100, 200, 500], 'max_features': ['log2', 'sqrt'], 'max_depth': [3, 10, 50]}],
                    [{'booster': ['gbtree', 'gblinear', 'dart'], 'learning_rate': [0.1, 0.05, 0.2], 'min_child_weight': [1], 'max_delta_step': [0]}]]

# Define the list of algorithms to tune
algorithms = [SVC(), DecisionTreeClassifier(), MLPClassifier(), GaussianNB(), LogisticRegression(), KNeighborsClassifier(), BaggingClassifier(), RandomForestClassifier(), AdaBoostClassifier(), GradientBoostingClassifier(), XGBClassifier()]

# Define the list of algorithm names
algorithm_names = ["SVC",
                   "DecisionTreeClassifier",
                   "MLPClassifier",
                   "GaussianNB",
                   "LogisticRegression",
                   "KNeighborsClassifier",
                   "BaggingClassifier",
                   "RandomForestClassifier",
                   "AdaBoostClassifier",
                   "GradientBoostingClassifier",
                   "XGBClassifier"]

# Perform grid search for each algorithm
for i in range(0, 11):
    print("################   %s   ################" % algorithm_names[i])

    # Define the scoring metric to use
    score = 'precision_macro'

    # Create a GridSearchCV object
    clf = model_selection.GridSearchCV(estimator=algorithms[i], param_grid=tuned_parameters[i], cv=5, scoring=score)

    # Fit the model to the training data
    clf.fit(X_train, y_train)

    # Print the best parameters found
    print("Best parameters set found on development set:")
    print(clf.best_params_)

    # Print the grid scores on the development set
    print("Grid scores on development set:")
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

    # Train the model on the full development set
    clf.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = clf.predict(X_test)

    # Print the classification report
    print("Detailed classification report:")
    print(metrics.classification_report(y_test, y_pred))

    # Print the confusion matrix
    print("Detailed confusion matrix:")
    print(metrics.confusion_matrix(y_test, y_pred))

    # Print the precision score
    print("Precision Score: \n")
    print(metrics.precision_score(y_test, y_pred))

# Print the completed message
print("\nGrid search completed!")


################   SVC   ################
Best parameters set found on development set:
{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
Grid scores on development set:
0.961 (+/-0.026) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.967 (+/-0.036) for {'C': 1, 'gamma': 0.001, 'kernel': 'linear'}
0.853 (+/-0.013) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.967 (+/-0.036) for {'C': 1, 'gamma': 0.0001, 'kernel': 'linear'}
0.977 (+/-0.035) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.960 (+/-0.062) for {'C': 10, 'gamma': 0.001, 'kernel': 'linear'}
0.961 (+/-0.026) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.960 (+/-0.062) for {'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
0.982 (+/-0.034) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.950 (+/-0.048) for {'C': 100, 'gamma': 0.001, 'kernel': 'linear'}
0.977 (+/-0.035) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.950 (+/-0.048) for {'C': 100, 'gamma': 0.0001, 'kernel': 'linear'}
0.975 (+/-0.045) for {'C': 1000, 'gamma': 0.

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptro

Best parameters set found on development set:
{'activation': 'tanh', 'alpha': 0.0005, 'hidden_layer_sizes': (10,), 'max_iter': 1000}
Grid scores on development set:
0.951 (+/-0.059) for {'activation': 'logistic', 'alpha': 0.0001, 'hidden_layer_sizes': (5,), 'max_iter': 200}
0.977 (+/-0.028) for {'activation': 'logistic', 'alpha': 0.0001, 'hidden_layer_sizes': (5,), 'max_iter': 1000}
0.977 (+/-0.030) for {'activation': 'logistic', 'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'max_iter': 200}
0.977 (+/-0.028) for {'activation': 'logistic', 'alpha': 0.0001, 'hidden_layer_sizes': (10,), 'max_iter': 1000}
0.948 (+/-0.050) for {'activation': 'logistic', 'alpha': 0.0005, 'hidden_layer_sizes': (5,), 'max_iter': 200}
0.980 (+/-0.027) for {'activation': 'logistic', 'alpha': 0.0005, 'hidden_layer_sizes': (5,), 'max_iter': 1000}
0.972 (+/-0.025) for {'activation': 'logistic', 'alpha': 0.0005, 'hidden_layer_sizes': (10,), 'max_iter': 200}
0.976 (+/-0.024) for {'activation': 'logistic', 'alpha': 0.

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptro

Detailed classification report:
              precision    recall  f1-score   support

           0       0.94      0.96      0.95        47
           1       0.97      0.96      0.96        67

    accuracy                           0.96       114
   macro avg       0.95      0.96      0.95       114
weighted avg       0.96      0.96      0.96       114

Detailed confusion matrix:
[[45  2]
 [ 3 64]]
Precision Score: 

0.9696969696969697
################   GaussianNB   ################
Best parameters set found on development set:
{}
Grid scores on development set:
0.948 (+/-0.045) for {}
Detailed classification report:
              precision    recall  f1-score   support

           0       0.88      0.89      0.88        47
           1       0.92      0.91      0.92        67

    accuracy                           0.90       114
   macro avg       0.90      0.90      0.90       114
weighted avg       0.90      0.90      0.90       114

Detailed confusion matrix:
[[42  5]
 [ 6 61]

...

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:56:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "max_delta_step", "min_child_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:56:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "max_delta_step", "min_child_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:56:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "max_delta_step", "min_child_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:56:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "max_delta_step", "min_child_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:56:11]

Best parameters set found on development set:
{'booster': 'gblinear', 'learning_rate': 0.05, 'max_delta_step': 0, 'min_child_weight': 1}
Grid scores on development set:
0.957 (+/-0.032) for {'booster': 'gbtree', 'learning_rate': 0.1, 'max_delta_step': 0, 'min_child_weight': 1}
0.956 (+/-0.038) for {'booster': 'gbtree', 'learning_rate': 0.05, 'max_delta_step': 0, 'min_child_weight': 1}
0.967 (+/-0.024) for {'booster': 'gbtree', 'learning_rate': 0.2, 'max_delta_step': 0, 'min_child_weight': 1}
0.967 (+/-0.036) for {'booster': 'gblinear', 'learning_rate': 0.1, 'max_delta_step': 0, 'min_child_weight': 1}
0.972 (+/-0.032) for {'booster': 'gblinear', 'learning_rate': 0.05, 'max_delta_step': 0, 'min_child_weight': 1}
0.965 (+/-0.049) for {'booster': 'gblinear', 'learning_rate': 0.2, 'max_delta_step': 0, 'min_child_weight': 1}
0.957 (+/-0.032) for {'booster': 'dart', 'learning_rate': 0.1, 'max_delta_step': 0, 'min_child_weight': 1}
0.956 (+/-0.038) for {'booster': 'dart', 'learning_rate': 0.05

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:56:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "max_delta_step", "min_child_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:56:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "max_delta_step", "min_child_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:56:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "max_delta_step", "min_child_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:56:29] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "max_delta_step", "min_child_weight" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:56:29]

Detailed classification report:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96        47
           1       0.97      0.97      0.97        67

    accuracy                           0.96       114
   macro avg       0.96      0.96      0.96       114
weighted avg       0.96      0.96      0.96       114

Detailed confusion matrix:
[[45  2]
 [ 2 65]]
Precision Score: 

0.9701492537313433

Grid search completed!


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:56:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "max_delta_step", "min_child_weight" } are not used.

  warnings.warn(smsg, UserWarning)
